In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime, timedelta
import re
import pytz

In [2]:

conn = psycopg2.connect(host=redacted, user=re,
                        password=redacted, database=redacted)

In [3]:
datedict={}

headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

utc_now = pytz.utc.localize(datetime.utcnow())
pst_now = utc_now.astimezone(pytz.timezone("America/Los_Angeles"))
yesterday=pst_now - timedelta(1)

yesteryear=str(yesterday.year)

yesday=str(yesterday.day)

yestermonth=str(yesterday.month)
    

url=requests.get('''https://www.mtggoldfish.com/tournament_searches/create?commit=Search&page='''+str(1)+'''&tournament_search%5Bdate_range%5D='''+yestermonth+'''%2F'''+yesday+'''%2F'''+yesteryear+'''+-+'''+yestermonth+'''%2F'''+yesday+'''%2F'''+yesteryear+'''&tournament_search%5Bformat%5D=pioneer&tournament_search%5Bname%5D=&utf8=%E2%9C%93''')
#url=requests.get('''https://www.mtggoldfish.com/tournament_searches/create?commit=Search&page='''+str(1)+'''&tournament_search%5Bdate_range%5D='''+yestermonth+'''%2F'''+str(28)+'''%2F'''+yesteryear+'''+-+'''+yestermonth+'''%2F'''+yesday+'''%2F'''+yesteryear+'''&tournament_search%5Bformat%5D=pioneer&tournament_search%5Bname%5D=&utf8=%E2%9C%93''')


rawtext=BeautifulSoup(url.text,'html.parser')

table=rawtext.find_all('tr')

for entry in table[1:]:


    linktocards = entry.find('a').get('href')
    entrydate = str(entry.find('td').contents[0])
    tourntitle = str(entry.find('a').contents[0])

    if entrydate not in datedict.keys():
        datedict[entrydate]={}

    if tourntitle not in datedict[entrydate].keys():
        datedict[entrydate][tourntitle]={}

    decklisturl=requests.get('https://www.mtggoldfish.com/'+str(linktocards))
    decktext=BeautifulSoup(decklisturl.text,'html.parser')

    deckstuff = decktext.find('table',attrs={'class':'table table-condensed table-bordered table-tournament'})
    if deckstuff==None:
        pass
    else:
        decklinks = deckstuff.find_all('a')

        for deck in decklinks:
            if str(deck.get('href')).lower().find('/deck/')!=-1:
                cardurl = requests.get('https://www.mtggoldfish.com'+str(deck.get('href')))
                cardtext = BeautifulSoup(cardurl.text,'html.parser')
                cardquantsarr=cardtext.find_all('td',attrs={'class':'deck-col-qty'})
                cardquantsarr=[int(quant.getText()) for quant in cardquantsarr]
                cardnamesintermed=[card for card in cardtext.find_all('td',attrs={'class':'deck-col-card'})]
                allcardnames = [cardinfo.find_all('a')[0].contents[0] for cardinfo in cardnamesintermed if len(cardinfo.find_all('a'))>0 and len(cardinfo.find_all('a')[0].contents)>0]
                cardandquant=zip(allcardnames,cardquantsarr)

                for cardnames,cardquants in cardandquant:
                    if cardnames in datedict[entrydate][tourntitle].keys():
                        datedict[entrydate][tourntitle][cardnames]+=cardquants
                    else:
                        datedict[entrydate][tourntitle][cardnames]=cardquants






In [4]:
# cur=conn.cursor()

# cur.execute('''CREATE TABLE tourninfo (id SERIAL PRIMARY KEY, cardname TEXT,entrydate TEXT,cardquant INT,tourntitle TEXT)''')

# conn.commit()

In [5]:
cur=conn.cursor()

for datekeys in datedict.keys():
    for tournkeys in datedict[datekeys].keys():
        for dictcards,dictquants in datedict[datekeys][tournkeys].items():
            cur.execute('''INSERT INTO tourninfo (cardname,entrydate,cardquant,tourntitle) VALUES (%s,%s,%s,%s)''',(dictcards,datekeys,dictquants,tournkeys))
            conn.commit()

In [6]:
cur.close()
conn.close()